# Combinatorial Encoding Tools  


**Author:** Joaquín Cerdá-Boluda  
**Model Reference:** *Permutational Cellular Automata: From Bitwise Reversibility to Quantum Dynamics*, Joaquín Cerdá-Boluda and Marta C. Mora  
**Version:** 1.0  
**Last update:** 6/11/2025  
[GitHub Repository](https://github.com/ximocerda/PermutationalCellularAutomata)

---





## Introduction

This Colab notebook provides an interactive environment to experiment with the Encoding tools needed to explore **Permutational Cellular Automata (PeCA)** model. PeCA is a discrete computational model where local update rules are based on **permutations** of bit segments, and interactions between cells are specified via configurable **connectivities**.

The notebook presents conversion functions between natural numbers, factoradic.  

Let’s start by implementing the foundational combinatorial machinery.

---

## Factoradic Numbers and Permutations: support functions

We begin with utility functions that allow conversion between:

- A natural number (integer)
- Its corresponding factoradic representation
- The associated permutation of N elements via the Lehmer code

In [1]:
import math

def int_to_factoradic(n: int, N: int) -> list[int]:
    """Convert integer to its factoradic representation with N digits."""
    digits = [0] * N
    for i in range(1, N):
        digits[-i-1] = n % (i+1)
        n //= (i+1)
    digits[-1] = n
    return digits

def factoradic_to_int(factoradic: list[int]) -> int:
    """Convert a factoradic list to its integer value."""
    return sum(d * math.factorial(i) for i, d in enumerate(reversed(factoradic)))

def factoradic_to_permutation(factoradic: list[int]) -> list[int]:
    """Convert a factoradic number (Lehmer code) to a permutation."""
    elements = list(range(len(factoradic)))
    return [elements.pop(i) for i in factoradic]

def permutation_to_factoradic(perm: list[int]) -> list[int]:
    """Convert a permutation to its Lehmer code (factoradic)."""
    elements = list(range(len(perm)))
    factoradic = []
    for x in perm:
        idx = elements.index(x)
        factoradic.append(idx)
        elements.pop(idx)
    return factoradic

def int_to_permutation(n: int, N: int) -> list[int]:
    """Map an integer to a permutation via factoradic encoding."""
    factoradic = int_to_factoradic(n, N)
    return factoradic_to_permutation(factoradic)

def permutation_to_int(perm: list[int]) -> int:
    """Convert a permutation to its corresponding integer via factoradic."""
    factoradic = permutation_to_factoradic(perm)
    return factoradic_to_int(factoradic)

In this cell we perform a quick test to see if conversion works propperly.

In [2]:
# ---------- QUICK TEST ----------
if __name__ == "__main__":
    num = 25461
    fact = int_to_factoradic(num, 8)
    perm = factoradic_to_permutation(fact)
    recovered = factoradic_to_int(fact)
    print(f"Original number: {num}")
    print(f"Factoradic: {fact}")
    print(f"Permutation: {perm}")
    print(f"Recovered number: {recovered}")


Original number: 25461
Factoradic: [5, 0, 2, 0, 3, 1, 1, 0]
Permutation: [5, 0, 3, 1, 7, 4, 6, 2]
Recovered number: 25461


# Interactive conversion: Integer → Factoradic and Permutation
Next, we introduce some interactive environments to perform conversions. The first one let us introduce an integer and expresses it in factoradic and permutation notation.

In [3]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

# Widgets
N_widget = widgets.IntSlider(value=4, min=1, max=12, description='N (size):')
int_input = widgets.BoundedIntText(value=0, min=0, max=23, description='Integer:')
compute_button = widgets.Button(description="Convert")
output = widgets.Output()

def update_bounds(*args):
    N = N_widget.value
    int_input.max = math.factorial(N) - 1
    if int_input.value > int_input.max:
        int_input.value = 0

N_widget.observe(update_bounds, names='value')

def on_convert_clicked(b):
    with output:
        clear_output()
        N = N_widget.value
        n = int_input.value
        factoradic = int_to_factoradic(n, N)
        permutation = int_to_permutation(n, N)
        perm_back = permutation_to_int(permutation)

        display(Markdown(f"### Results for N = {N}, Integer = {n}"))
        display(Markdown(f"**Factoradic**: {factoradic}"))
        display(Markdown(f"**Permutation**: {permutation}"))
        display(Markdown(f"**Back to Integer** (via perm → fact → int): `{perm_back}`"))

compute_button.on_click(on_convert_clicked)

# Display UI
display(N_widget, int_input, compute_button, output)


IntSlider(value=4, description='N (size):', max=12, min=1)

BoundedIntText(value=0, description='Integer:', max=23)

Button(description='Convert', style=ButtonStyle())

Output()

# Interactive full conversion: Integer ↔ Factoradic ↔ Permutation
This cell allolws introducing any description to obtain the other two.

In [7]:
from IPython.display import display, Markdown
import ipywidgets as widgets

# --- Factoradic & Permutation Utilities ---
def int_to_factoradic(n: int, N: int) -> list[int]:
    """Convert an integer to an N-digit factoradic number."""
    f = [0] * N
    for i in range(1, N):
        f[N - 1 - i] = n % (i + 1)
        n //= (i + 1)
    return f

def factoradic_to_int(f: list[int]) -> int:
    """Convert a factoradic number to integer."""
    return sum(d * factorial(i) for i, d in enumerate(reversed(f)))

def int_to_permutation(n: int, N: int) -> list[int]:
    """Convert an integer to a permutation using its Lehmer code."""
    factoradic = int_to_factoradic(n, N)
    elements = list(range(N))
    perm = []
    for d in factoradic:
        perm.append(elements.pop(d))
    return perm

def permutation_to_int(perm: list[int]) -> int:
    """Convert a permutation to integer using its Lehmer code."""
    N = len(perm)
    elements = list(range(N))
    code = []
    for x in perm:
        i = elements.index(x)
        code.append(i)
        elements.pop(i)
    return factoradic_to_int(code)

from math import factorial

# --- Widgets ---
n_input = widgets.BoundedIntText(value=0, min=0, max=1000000, description="Integer:")
factoradic_input = widgets.Text(value="", description="Factoradic:")
permutation_input = widgets.Text(value="", description="Permutation:")
N_select = widgets.BoundedIntText(value=4, min=1, max=15, description="N:")

# --- Synchronization Logic ---
def update_from_integer(change):
    N = N_select.value
    n = n_input.value
    try:
        f = int_to_factoradic(n, N)
        p = int_to_permutation(n, N)
        factoradic_input.value = " ".join(map(str, f))
        permutation_input.value = " ".join(map(str, p))
    except Exception as e:
        factoradic_input.value = f"Error: {e}"
        permutation_input.value = ""

def update_from_factoradic(change):
    N = N_select.value
    try:
        f = list(map(int, factoradic_input.value.strip().split()))
        if len(f) != N:
            raise ValueError("Wrong length")
        n = factoradic_to_int(f)
        p = int_to_permutation(n, N)
        n_input.value = n
        permutation_input.value = " ".join(map(str, p))
    except Exception as e:
        permutation_input.value = f"Error: {e}"


def update_from_permutation(change):
    N = N_select.value
    try:
        p = list(map(int, permutation_input.value.strip().split()))
        if sorted(p) != list(range(N)):
            raise ValueError("Not a valid permutation")
        n = permutation_to_int(p)
        f = int_to_factoradic(n, N)
        n_input.value = n
        factoradic_input.value = " ".join(map(str, f))
    except Exception as e:
        factoradic_input.value = f"Error: {e}"

# --- Observers ---
n_input.observe(update_from_integer, names='value')
factoradic_input.observe(update_from_factoradic, names='value')
permutation_input.observe(update_from_permutation, names='value')

# --- Display ---
display(Markdown("## Convert Between Integer, Factoradic, and Permutation"))
display(N_select, n_input, factoradic_input, permutation_input)

# Trigger initial update
update_from_integer(None)


## Convert Between Integer, Factoradic, and Permutation

BoundedIntText(value=4, description='N:', max=15, min=1)

BoundedIntText(value=0, description='Integer:', max=1000000)

Text(value='', description='Factoradic:')

Text(value='', description='Permutation:')

# Interactive conversion: Permutation ↔ Cyclic
This cell allows alternating between permutation and cycle notation.

In [6]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output

def permutation_to_cycles(perm):
    """Convert a permutation to disjoint cycle notation."""
    n = len(perm)
    seen = [False] * n
    cycles = []
    for i in range(n):
        if not seen[i]:
            cycle = []
            j = i
            while not seen[j]:
                seen[j] = True
                cycle.append(j)
                j = perm[j]
            if len(cycle) > 1:
                cycles.append(cycle)
    return ''.join(f"({' '.join(map(str, c))})" for c in cycles) or "()"

def cycles_to_permutation(cycle_str, n):
    """Convert a cycle string (like '(0 2)(1 3)') to a permutation list."""
    import re
    perm = list(range(n))
    tokens = re.findall(r'\((.*?)\)', cycle_str)
    for tok in tokens:
        elems = list(map(int, tok.strip().split()))
        for i in range(len(elems)):
            perm[elems[i]] = elems[(i + 1) % len(elems)]
    return perm

# Widgets
N_input = widgets.BoundedIntText(value=5, min=2, max=12, description="Size N:")
perm_input = widgets.Text(value="0 2 1 4 3", description="Permutation:")
cycles_input = widgets.Text(value="(1 2)(3 4)", description="Cycle Notation:")

# Output area
out = widgets.Output()

def update_from_perm(change=None):
    try:
        perm = list(map(int, perm_input.value.strip().split()))
        cycles = permutation_to_cycles(perm)
        cycles_input.value = cycles
    except Exception as e:
        with out:
            clear_output()
            print(f"Error in permutation input: {e}")

def update_from_cycles(change=None):
    try:
        n = N_input.value
        perm = cycles_to_permutation(cycles_input.value.strip(), n)
        perm_input.value = ' '.join(map(str, perm))
    except Exception as e:
        with out:
            clear_output()
            print(f"Error in cycle input: {e}")

def update_size(change=None):
    update_from_perm()

perm_input.observe(update_from_perm, names='value')
cycles_input.observe(update_from_cycles, names='value')
N_input.observe(update_size, names='value')

display(Markdown("### Permutation ↔ Cycle Notation Converter"))
display(N_input, perm_input, cycles_input, out)


### Permutation ↔ Cycle Notation Converter

BoundedIntText(value=5, description='Size N:', max=12, min=2)

Text(value='0 2 1 4 3', description='Permutation:')

Text(value='(1 2)(3 4)', description='Cycle Notation:')

Output()